# Plan

When we hear the name Boston, we immediately think of either MIT or Haravad.A study made in 2017 listed Boston among top ten cities for students. In this city reside probably the best minds of the world. So what if you are a student and want to buy an apartment or rent a hostel ? Which neighborhood in the city will be the best for residing? That's the answer that we will try to learn through the outputs of our algorithm.

## Dataset

Before using Foursquare API to locate the popular venues in the neighborhood, we need to make a dataframe of neighborhoods of Boston and their corresponding longitudes and latitudes. The neigborhood dataset can be collected from https://data.boston.gov/dataset/boston-neighborhoods. Then this datasets need to be cleaned by removing unnecessary columns and by adding columns of corresponding longitudes and latitudes so that we can apply our algorithm to it.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
Latitudes=[42.28321,42.30974,42.32962,42.34273,42.34896,42.35049,42.35006,42.36470,42.31520,42.33881,42.35026,42.37021,42.37821,42.36436,42.35880,42.35571,42.34287,42.34635,42.27976,42.25653,42.27714,42.30163,42.33814,42.35390]
Longitudes=[-71.12703,-71.11514,-71.10623,-71.10994,-71.06983,-71.05795,-71.06241,-71.05423,-71.09137,-71.07652,-71.08098,-71.03886,-71.06021,-71.06612,-71.07074,-71.05721,-71.10029,-71.16268,-71.16268,-71.12406,-71.09137,-71.06760,-71.04758,-71.13371]

In [3]:
import pandas as pd
boston_neighborhoods=pd.read_csv("Boston.csv")

In [4]:
boston_neighborhoods.head()

,OBJECTID,Name,Acres,Neighborhood_ID,SqMiles,ShapeSTArea,ShapeSTLength
0,27,Roslindale,1605.568238,15,2.51,6.993827e+07,53563.912600
1,28,Jamaica Plain,2519.245394,11,3.94,1.097379e+08,56349.937160
2,29,Mission Hill,350.853564,13,0.55,1.528312e+07,17918.724110
3,30,Longwood,188.611947,28,0.29,8.215904e+06,11908.757150
4,31,Bay Village,26.539839,33,0.04,1.156071e+06,4650.635493


In [5]:
boston = boston_neighborhoods.drop(['OBJECTID','Acres','Neighborhood_ID','ShapeSTArea','ShapeSTLength','SqMiles'],axis=1)
boston.head()

,Name
0,Roslindale
1,Jamaica Plain
2,Mission Hill
3,Longwood
4,Bay Village


In [6]:
boston['Longitudes']=pd.Series(Longitudes)
boston['Latitudes']=pd.Series(Latitudes)
boston.rename(columns={'Name':'Neighborhood'},inplace=True)
boston


,Neighborhood,Longitudes,Latitudes
0,Roslindale,-71.12703,42.28321
1,Jamaica Plain,-71.11514,42.30974
2,Mission Hill,-71.10623,42.32962
3,Longwood,-71.10994,42.34273
4,Bay Village,-71.06983,42.34896
5,Leather District,-71.05795,42.35049
6,Chinatown,-71.06241,42.35006
7,North End,-71.05423,42.36470
8,Roxbury,-71.09137,42.31520
9,South End,-71.07652,42.33881


## Exploring and Analyzing Boston

In [7]:
#Longitude and latitude of Boston
address = 'Boston, MA'

geolocator = Nominatim(user_agent="boston_viewer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Boston are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Boston are 42.3602534, -71.0582912.


## Map of Neighborhoods of Boston

In [8]:
# create map of Boston using latitude and longitude values
map_boston = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(boston['Latitudes'], boston['Longitudes'], boston['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_boston)  
    
map_boston

## Making the foursquare  API Call

In [9]:
#Forsquare credentials and version
CLIENT_ID = 'ZHHQ1UIQAHYSW4WHU142SPILV5K2V5VJ2WZ3FCBJMDKIUKN1' # your Foursquare ID
CLIENT_SECRET = 'YMQFNQSSEV0UMNZHRF1ORQVPZLSTJFUPBO2GY4FW5TA0OJ2Q' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZHHQ1UIQAHYSW4WHU142SPILV5K2V5VJ2WZ3FCBJMDKIUKN1
CLIENT_SECRET:YMQFNQSSEV0UMNZHRF1ORQVPZLSTJFUPBO2GY4FW5TA0OJ2Q


In [10]:
#Exploring the first neighborhood
boston.loc[0, 'Neighborhood']

'Roslindale'

In [11]:
neighborhood_latitude = boston.loc[0, 'Latitudes'] # neighborhood latitude value
neighborhood_longitude = boston.loc[0, 'Longitudes'] # neighborhood longitude value

neighborhood_name = boston.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Roslindale are 42.28321, -71.12703.


In [12]:
#Top 100 venues for Roslindale
LIMIT=100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/explore?&client_id=ZHHQ1UIQAHYSW4WHU142SPILV5K2V5VJ2WZ3FCBJMDKIUKN1&client_secret=YMQFNQSSEV0UMNZHRF1ORQVPZLSTJFUPBO2GY4FW5TA0OJ2Q&v=20180605&ll=42.28321,-71.12703&radius=500&limit=100'

In [13]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f1d9a1913a607527cc66478'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Roslindale',
  'headerFullLocation': 'Roslindale, Boston',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 23,
  'suggestedBounds': {'ne': {'lat': 42.2877100045, 'lng': -71.12095885811765},
   'sw': {'lat': 42.278709995499995, 'lng': -71.13310114188236}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b12c6b0f964a520eb8d23e3',
       'name': 'Fornax Bread Company',
       'location': {'address': '27 Corinth St',
        'lat': 42.28617090302737,
        'lng': -71.12976037438125,
        'labeledLatLngs': [{'labe

In [14]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [15]:
#Structuring into a pandas dataframe
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Fornax Bread Company,Bakery,42.286171,-71.129760
1,Roslindale Village Farmers Market,Farmers Market,42.286534,-71.128509
2,Roslindale Square,Plaza,42.285815,-71.129304
3,Sophia's Grotto,Italian Restaurant,42.286616,-71.129869
4,Effie's Kitchen,Greek Restaurant,42.285899,-71.128741


In [16]:
#The number of venues returned for this neighborhood 
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

23 venues were returned by Foursquare.


In [17]:
#Repeat the process for all neighborhoods in Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
#Venues in Boston
Boston_venues = getNearbyVenues(names=boston['Neighborhood'],
                                   latitudes=boston['Latitudes'],
                                   longitudes=boston['Longitudes'])



Roslindale
Jamaica Plain
Mission Hill
Longwood
Bay Village
Leather District
Chinatown
North End
Roxbury
South End
Back Bay
East Boston
Charlestown
West End
Beacon Hill
Downtown
Fenway
Brighton
West Roxbury
Hyde Park
Mattapan
Dorchester
South Boston
Allston


In [19]:
#Total Venues in Boston
print(Boston_venues.shape)
Boston_venues.head()

(1088, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Roslindale,42.28321,-71.12703,Fornax Bread Company,42.286171,-71.129760,Bakery
1,Roslindale,42.28321,-71.12703,Roslindale Village Farmers Market,42.286534,-71.128509,Farmers Market
2,Roslindale,42.28321,-71.12703,Roslindale Square,42.285815,-71.129304,Plaza
3,Roslindale,42.28321,-71.12703,Sophia's Grotto,42.286616,-71.129869,Italian Restaurant
4,Roslindale,42.28321,-71.12703,Effie's Kitchen,42.285899,-71.128741,Greek Restaurant


In [20]:
#Group Venues by neighborhood and count them
Boston_venues.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allston,85,85,85,85,85,85
Back Bay,100,100,100,100,100,100
Bay Village,63,63,63,63,63,63
Beacon Hill,37,37,37,37,37,37
Brighton,16,16,16,16,16,16
Charlestown,22,22,22,22,22,22
Chinatown,78,78,78,78,78,78
Dorchester,3,3,3,3,3,3
Downtown,98,98,98,98,98,98


In [21]:
print('There are {} uniques categories.'.format(len(Boston_venues['Venue Category'].unique())))

There are 210 uniques categories.


In [22]:
#Analyze each neighborhood
# one hot encoding
boston_onehot = pd.get_dummies(Boston_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
boston_onehot['Neighborhood'] = Boston_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [boston_onehot.columns[-1]] + list(boston_onehot.columns[:-1])
boston_onehot = boston_onehot[fixed_columns]

boston_onehot.head()

,Zoo Exhibit,ATM,Accessories Store,American Restaurant,Arepa Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Service,Café,Caribbean Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Cycle Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Food,Food & Drink Shop,Food Court,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,National Park,Neighborhood,New American Restaurant,Noodle House,Office,Opera House,Optical Shop,Other Repair Shop,Outdoor Sculpture,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Planetarium,Playground,Plaza,Post Office,Pub,Ramen Restaurant,Record Shop,Resort,Restaurant,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tour Provider,Track Stadium,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Roslindale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Roslindale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [23]:
boston_onehot.shape

(1088, 210)

In [24]:
boston_grouped = boston_onehot.groupby('Neighborhood').mean().reset_index()
boston_grouped

,Neighborhood,Zoo Exhibit,ATM,Accessories Store,American Restaurant,Arepa Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Business Service,Café,Caribbean Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Cycle Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Food,Food & Drink Shop,Food Court,Food Truck,Forest,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Health & Beauty Service,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,National Park,New American Restaurant,Noodle House,Office,Opera House,Optical Shop,Other Repair Shop,Outdoor Sculpture,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Planetarium,Playground,Plaza,Post Office,Pub,Ramen Restaurant,Record Shop,Resort,Restaurant,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Ski Chalet,Smoke Shop,Smoothie Shop,Snack Place,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tour Provider,Track Stadium,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Allston,0.000000,0.000000,0.000000,0.000000,0.000000,0.011765,0.000000,0.023529,0.000000,0.000000,0.000000,0.000000,0.035294,0.011765,0.035294,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.011765,0.000000,0.000000,0.000000,0.000000,0.000000,0.023529,0.011765,0.011765,0.000000,0.000000,0.00,0.011765,0.000000,0.000000,0.000000,0.035294,0.00,0.000000,0.000000,0.000000,0.011765,0.000000,0.000000,0.011765,0.00,0.00000,0.00,0.035294,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.011765,0.000000,0.000000,0.000000,0.011765,0.023529,0.011765,0.011765,0.011765,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023529,0.011765,0.000000,0.000000,0.000000,0.000000,0.023529,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011765,0.011765,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011765,0.000000,0.011765,0.000000,0.000000,0.000000,0.011765,0.011765,0.000000,0.011765,0.011765,0.000000,0.000000,0.000000,0.000000,0.082353,0.0000,0.011765,0.000000,0.000000,0.023529,0.000000,0.000000,0.000000,0.000000,0.00

In [25]:
boston_grouped.shape

(24, 210)

In [26]:
#Print each neihborhood with ten most common venues
num_top_venues = 10

for hood in boston_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = boston_grouped[boston_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allston----
                venue  freq
0   Korean Restaurant  0.08
1         Pizza Place  0.05
2   Convenience Store  0.04
3     Thai Restaurant  0.04
4    Sushi Restaurant  0.04
5                 Bar  0.04
6           Rock Club  0.04
7              Bakery  0.04
8  Chinese Restaurant  0.04
9          Donut Shop  0.02


----Back Bay----
                 venue  freq
0          Coffee Shop  0.07
1       Clothing Store  0.05
2   Seafood Restaurant  0.05
3       Ice Cream Shop  0.05
4  American Restaurant  0.05
5   Italian Restaurant  0.04
6                Hotel  0.03
7        Women's Store  0.02
8                  Bar  0.02
9     Department Store  0.02


----Bay Village----
               venue  freq
0                Spa  0.06
1            Theater  0.06
2     Sandwich Place  0.05
3     Clothing Store  0.05
4              Hotel  0.05
5                Gym  0.03
6  French Restaurant  0.03
7             Bakery  0.03
8       Gourmet Shop  0.03
9        Pizza Place  0.03


----Beacon Hill--

In [27]:
#Create new dataframe and display top ten venues for each neighborhood
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
num_top_venues = 15

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = boston_grouped['Neighborhood']

for ind in np.arange(boston_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(boston_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Allston,Korean Restaurant,Pizza Place,Thai Restaurant,Bar,Sushi Restaurant,Rock Club,Convenience Store,Chinese Restaurant,Bakery,Donut Shop,Bubble Tea Shop,Liquor Store,Mexican Restaurant,Fried Chicken Joint,Asian Restaurant
1,Back Bay,Coffee Shop,Clothing Store,American Restaurant,Ice Cream Shop,Seafood Restaurant,Italian Restaurant,Hotel,Shopping Mall,Plaza,Restaurant,Grocery Store,Mediterranean Restaurant,Sporting Goods Shop,New American Restaurant,Steakhouse
2,Bay Village,Spa,Theater,Sandwich Place,Clothing Store,Hotel,French Restaurant,Pizza Place,Italian Restaurant,Bakery,Gourmet Shop,American Restaurant,Wine Shop,Gym,Pilates Studio,Cupcake Shop
3,Beacon Hill,Playground,Italian Restaurant,Pizza Place,Hotel Bar,Gourmet Shop,Breakfast Spot,General Entertainment,Bistro,Pub,French Restaurant,Dessert Shop,Coffee Shop,Mexican Restaurant,Café,Music Venue
4,Brighton,Pizza Place,Park,Coffee Shop,Café,Supplement Shop,Sushi Restaurant,Liquor Store,Chinese Restaurant,Mobile Phone Shop,Lake,Salon / Barbershop,Convenience Store,Wings Joint,Dumpling Restaurant,Food & Drink Shop


### Clustering the neighborhod into 5 neighborhoods

### Run k-means to cluster the neighborhoods into five clusters

In [29]:
# set number of clusters
kclusters = 5

boston_grouped_clustering = boston_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(boston_grouped_clustering)

# check cluster labels generated for each row in the dataframe
len(kmeans.labels_) 

24

In [30]:
#Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

boston_merged = boston

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
boston_merged = boston_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

#boston_merged.head() # check the last columns!
boston_merged['Cluster Labels']=boston_merged['Cluster Labels'].apply(lambda x: int(x))
boston_merged.head()

,Neighborhood,Longitudes,Latitudes,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Roslindale,-71.12703,42.28321,0,American Restaurant,Italian Restaurant,Indian Restaurant,Greek Restaurant,Deli / Bodega,Plaza,Pub,Restaurant,Discount Store,Coffee Shop,Sandwich Place,Chinese Restaurant,Cheese Shop,Farmers Market,Forest
1,Jamaica Plain,-71.11514,42.30974,0,Bakery,Yoga Studio,Accessories Store,Art Gallery,Pizza Place,Donut Shop,Coffee Shop,Thrift / Vintage Store,Deli / Bodega,Liquor Store,Burrito Place,Seafood Restaurant,Sandwich Place,Mexican Restaurant,Record Shop
2,Mission Hill,-71.10623,42.32962,0,American Restaurant,Liquor Store,Indian Restaurant,Bank,Ice Cream Shop,Greek Restaurant,Metro Station,Park,Pharmacy,Pizza Place,Caribbean Restaurant,Burrito Place,Supermarket,Sushi Restaurant,Hotel
3,Longwood,-71.10994,42.34273,0,Park,Metro Station,Bakery,Diner,Coffee Shop,Pub,Chinese Restaurant,Café,Sandwich Place,Tapas Restaurant,Pharmacy,Shipping Store,Bubble Tea Shop,Spa,Sushi Restaurant
4,Bay Village,-71.06983,42.34896,0,Spa,Theater,Sandwich Place,Clothing Store,Hotel,French Restaurant,Pizza Place,Italian Restaurant,Bakery,Gourmet Shop,American Restaurant,Wine Shop,Gym,Pilates Studio,Cupcake Shop


### Lets visualize the resulting clusters

In [31]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(boston_merged['Latitudes'], boston_merged['Longitudes'], boston_merged['Neighborhood'], boston_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters
#### Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.



### Cluster 1

In [32]:
boston_merged[boston_merged['Cluster Labels'] == 0].reset_index(drop=True)

,Neighborhood,Longitudes,Latitudes,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Roslindale,-71.12703,42.28321,0,American Restaurant,Italian Restaurant,Indian Restaurant,Greek Restaurant,Deli / Bodega,Plaza,Pub,Restaurant,Discount Store,Coffee Shop,Sandwich Place,Chinese Restaurant,Cheese Shop,Farmers Market,Forest
1,Jamaica Plain,-71.11514,42.30974,0,Bakery,Yoga Studio,Accessories Store,Art Gallery,Pizza Place,Donut Shop,Coffee Shop,Thrift / Vintage Store,Deli / Bodega,Liquor Store,Burrito Place,Seafood Restaurant,Sandwich Place,Mexican Restaurant,Record Shop
2,Mission Hill,-71.10623,42.32962,0,American Restaurant,Liquor Store,Indian Restaurant,Bank,Ice Cream Shop,Greek Restaurant,Metro Station,Park,Pharmacy,Pizza Place,Caribbean Restaurant,Burrito Place,Supermarket,Sushi Restaurant,Hotel
3,Longwood,-71.10994,42.34273,0,Park,Metro Station,Bakery,Diner,Coffee Shop,Pub,Chinese Restaurant,Café,Sandwich Place,Tapas Restaurant,Pharmacy,Shipping Store,Bubble Tea Shop,Spa,Sushi Restaurant
4,Bay Village,-71.06983,42.34896,0,Spa,Theater,Sandwich Place,Clothing Store,Hotel,French Restaurant,Pizza Place,Italian Restaurant,Bakery,Gourmet Shop,American Restaurant,Wine Shop,Gym,Pilates Studio,Cupcake Shop
5,Leather District,-71.05795,42.35049,0,Chinese Restaurant,Bakery,Asian Restaurant,Sandwich Place,Coffee Shop,Sushi Restaurant,Dessert Shop,Noodle House,Food Truck,Bubble Tea Shop,Vegetarian / Vegan Restaurant,Café,Hotpot Restaurant,American Restaurant,Wine Bar
6,Chinatown,-71.06241,42.35006,0,Chinese Restaurant,Asian Restaurant,Bakery,Bubble Tea Shop,Performing Arts Venue,Sushi Restaurant,Coffee Shop,Sandwich Place,Theater,Restaurant,Seafood Restaurant,Dive Bar,Dim Sum Restaurant,Gym / Fitness Center,Café
7,North End,-71.05423,42.36470,0,Italian Restaurant,Park,Pizza Place,Seafood Restaurant,Coffee Shop,Café,Bakery,Sandwich Place,Sports Bar,Historic Site,Pub,Grocery Store,Harbor / Marina,Ski Chalet,Skating Rink
8,South End,-71.07652,42.33881,0,Italian Restaurant,Coffee Shop,Mediterranean Restaurant,Gym / Fitness Center,Tapas Restaurant,Sandwich Place,Park,Bakery,French Restaurant,Café,Deli / Bodega,Yoga Studio,Mexican Restaurant,Sushi Restaurant,Garden Center
9,Back Bay,-71.08098,42.35026,0,Coffee Shop,Clothing Store,American Restaurant,Ice Cream Shop,Seafood Restaurant,Italian Restaurant,Hotel,Shopping Mall,Plaza,Restaurant,Grocery Store,Mediterranean Restaurant,Sporting Goods Shop,New American Restaurant,Steakhouse


### Cluster 2

In [33]:
boston_merged[boston_merged['Cluster Labels'] == 1].reset_index(drop=True)

,Neighborhood,Longitudes,Latitudes,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Roxbury,-71.09137,42.3152,1,Convenience Store,Museum,Cosmetics Shop,French Restaurant,Forest,Food Truck,Food Court,Food & Drink Shop,Food,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Electronics Store,Dumpling Restaurant,Donut Shop


### Cluster 3

In [34]:
boston_merged[boston_merged['Cluster Labels'] == 2].reset_index(drop=True)

,Neighborhood,Longitudes,Latitudes,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Mattapan,-71.09137,42.27714,2,Ice Cream Shop,Restaurant,Donut Shop,Forest,Food Truck,Food Court,Food & Drink Shop,Food,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Electronics Store,Dumpling Restaurant,Dog Run,Fried Chicken Joint


### Cluster 4

In [35]:
boston_merged[boston_merged['Cluster Labels'] == 3].reset_index(drop=True)

,Neighborhood,Longitudes,Latitudes,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,West Roxbury,-71.16268,42.27976,3,Grocery Store,Pub,Automotive Shop,Pizza Place,Pharmacy,Salon / Barbershop,Donut Shop,Gas Station,Liquor Store,Bank,Gym / Fitness Center,Convenience Store,Train Station,Harbor / Marina,Hockey Arena


### Cluster 5

In [36]:
boston_merged[boston_merged['Cluster Labels'] == 4].reset_index(drop=True)

,Neighborhood,Longitudes,Latitudes,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,Dorchester,-71.0676,42.30163,4,Pizza Place,Food,Caribbean Restaurant,Yoga Studio,Dumpling Restaurant,Forest,Food Truck,Food Court,Food & Drink Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Electronics Store,Donut Shop,Fried Chicken Joint


### Conclusion

We can see that while most of the neighborhoods are saturated with restuarants, the most convenient place to live is the only neighborhood in cluster 4 that is West Roxbury. The neighborhood's most common venues include places like grocery store, pub, pharmacy, bank gym etc. which are essentials of a living place. These places are in walking distance and hence it is very feasible to reside in this neighborhood.